# Bayesian Classification for Machine Learning for Computational Linguistics

## Using token probabilities for classification

**(C) 2017-2025 by [Damir Cavar](http://damir.cavar.me/)**

**Download:** This and various other Jupyter notebooks are available from my [GitHub repo](https://github.com/dcavar/python-tutorial-notebooks).

**Version:** 1.7, September 2025

**License:** [Creative Commons Attribution-ShareAlike 4.0 International License](https://creativecommons.org/licenses/by-sa/4.0/) ([CA BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/))

This is a tutorial related to the discussion of a Bayesian classifier in the textbook [Machine Learning: The Art and Science of Algorithms that Make Sense of Data](https://www.cs.bris.ac.uk/~flach/mlbook/) by [Peter Flach](https://www.cs.bris.ac.uk/~flach/).

This tutorial was developed as part of my course material for the course Machine Learning for Computational Linguistics in the [Computational Linguistics Program](http://cl.indiana.edu/) of the [Department of Linguistics](http://www.indiana.edu/~lingdept/) at [Indiana University](https://www.indiana.edu/).

**Prerequisites:**

In [ ]:
!pip install -U nltk

## Creating a Training Corpus

Assume that we have a set of e-mails that are annotated as *spam* or *ham*, as described in the textbook.

There are $4$ e-mails labeled *ham* and $1$ e-mail is labeled *spam*, that is we have a total of $5$ texts in our corpus.

If we would randomly pick an e-mail from the collection, the probability that we pick a spam e-mail would be $1 / 5$.

Spam emails might differ from ham e-mails just in some words. Here is a sample email constructed with typical keywords:

In [1]:
spam = [ """Our medicine cures baldness. No diagnostics needed.
            We guarantee Fast Viagra delivery. V1agra
            We can provide Human growth hormone. The cheapest Life
            Insurance with us. You can Lose weight with this treatment.
            Our Medicine now and No medical exams necessary.
            Our Online pharmacy is the best.  This cream Removes
            wrinkles and Reverses aging.
            One treatment and you will Stop snoring.  We sell Valium
            and Viagra.
            Our Vicodin will help with Weight loss. Cheap Xanax.""",
        """ We have detected suspicious activity on your account.
        To prevent permanent suspension, you must immediately verify
        your identity by clicking the link below and updating your payment information. 
        http://www.micros0ft-teams-support.com/verify 
        Failure to do so within 12 hours will result in your account being locked.
        Thank you for your prompt attention to this matter.""" ]

The data structure above is a list of strings that contains only one string. The triple-double-quotes mark multi-line text. We can output the size of the variable *spam* this way:

In [2]:
print(len(spam))

2


We can create a list of *ham* mails in a similar way:

In [3]:
ham = [ """Hi Hans, hope to see you soon at our family party.
           When will you arrive.
           All the best to the family.
           Sue""",
      """Dear Ata,
         did you receive my last email related to the car insurance
         offer? I would be happy to discuss the details with you.
         Please give me a call, if you have any questions.
         John Smith
         Super Car Insurance""",
      """Hi everyone:
         This is just a gentle reminder of today's first 2017 SLS
         Colloquium, from 2.30 to 4.00 pm, in Ballantine 103.
         Rodica Frimu will present a job talk entitled "What is
         so tricky in subject-verb agreement?". The text of the
         abstract is below.
         If you would like to present something during the Spring,
         please let me know.
         The current online schedule with updated title
         information and abstracts is available under:
         http://www.iub.edu/~psyling/SLSColloquium/Spring2017.html
         See you soon,
         Peter""",
      """Dear Friends,
         As our first event of 2017, the Polish Studies Center
         presents an evening with artist and filmmaker Wojtek Sawa.
         Please join us on JANUARY 26, 2017 from 5:30 p.m. to
         7:30 p.m. in the Global and International Studies
         Building room 1100 for a presentation by Wojtek Sawa
         on his interactive  installation art piece The Wall
         Speaks–Voices of the Unheard. A reception will follow
         the event where you will have a chance to meet the artist
         and discuss his work.
         Best,""",
         """Hi Sue, yes I will be there. Just got back from my trip.
           See you soon. Hans""",
           """Dear John, yes I got your email. Please call me
           tomorrow. Ata""",
           """Hey, Peter, I was wondering whether you have some suggestion related to
            my research topic. Can we meet next week? I will be in Bloomington
            all of the week.""",]

The ham-mail list contains $4$ e-mails:

In [4]:
print(len(ham))

7


We can access a particular e-mail via index from either spam or ham:

In [5]:
print(spam[0])

Our medicine cures baldness. No diagnostics needed.
            We guarantee Fast Viagra delivery. V1agra
            We can provide Human growth hormone. The cheapest Life
            Insurance with us. You can Lose weight with this treatment.
            Our Medicine now and No medical exams necessary.
            Our Online pharmacy is the best.  This cream Removes
            wrinkles and Reverses aging.
            One treatment and you will Stop snoring.  We sell Valium
            and Viagra.
            Our Vicodin will help with Weight loss. Cheap Xanax.


In [6]:
print(ham[3])

Dear Friends,
         As our first event of 2017, the Polish Studies Center
         presents an evening with artist and filmmaker Wojtek Sawa.
         Please join us on JANUARY 26, 2017 from 5:30 p.m. to
         7:30 p.m. in the Global and International Studies
         Building room 1100 for a presentation by Wojtek Sawa
         on his interactive  installation art piece The Wall
         Speaks–Voices of the Unheard. A reception will follow
         the event where you will have a chance to meet the artist
         and discuss his work.
         Best,


We can lower-case the email using the string *lower* function:

In [7]:
print(ham[3].lower())

dear friends,
         as our first event of 2017, the polish studies center
         presents an evening with artist and filmmaker wojtek sawa.
         please join us on january 26, 2017 from 5:30 p.m. to
         7:30 p.m. in the global and international studies
         building room 1100 for a presentation by wojtek sawa
         on his interactive  installation art piece the wall
         speaks–voices of the unheard. a reception will follow
         the event where you will have a chance to meet the artist
         and discuss his work.
         best,


We can loop over all e-mails in spam or ham and lower-case the content:

In [8]:
for text in ham:
    print(text.lower())

hi hans, hope to see you soon at our family party.
           when will you arrive.
           all the best to the family.
           sue
dear ata,
         did you receive my last email related to the car insurance
         offer? i would be happy to discuss the details with you.
         please give me a call, if you have any questions.
         john smith
         super car insurance
hi everyone:
         this is just a gentle reminder of today's first 2017 sls
         colloquium, from 2.30 to 4.00 pm, in ballantine 103.
         rodica frimu will present a job talk entitled "what is
         so tricky in subject-verb agreement?". the text of the
         abstract is below.
         if you would like to present something during the spring,
         please let me know.
         the current online schedule with updated title
         information and abstracts is available under:
         http://www.iub.edu/~psyling/slscolloquium/spring2017.html
         see you soon,
         peter
d

We can use the tokenizer from NLTK to tokenize the lower-cased text into single tokens (words and punctuation marks):

In [9]:
from nltk import word_tokenize

print(word_tokenize(ham[0].lower()))

['hi', 'hans', ',', 'hope', 'to', 'see', 'you', 'soon', 'at', 'our', 'family', 'party', '.', 'when', 'will', 'you', 'arrive', '.', 'all', 'the', 'best', 'to', 'the', 'family', '.', 'sue']


We can count the numer of tokens and types in lower-cased text:

In [10]:
from collections import Counter

myCounts = Counter(word_tokenize("This is a test. Will this test teach us how to count tokens?".lower()))

print(myCounts)
print("number of  types:", len(myCounts))
print("number of tokens:", sum(myCounts.values()))

Counter({'this': 2, 'test': 2, 'is': 1, 'a': 1, '.': 1, 'will': 1, 'teach': 1, 'us': 1, 'how': 1, 'to': 1, 'count': 1, 'tokens': 1, '?': 1})
number of  types: 13
number of tokens: 15


Now we can create a frequency profile of ham and spam words given the two text collections:

In [11]:
hamFP = Counter()
spamFP = Counter()

for text in spam:
    spamFP.update(word_tokenize(text.lower()))

for text in ham:
    hamFP.update(word_tokenize(text.lower()))

print("Ham:\n",  hamFP)
print("-" * 30)
print("Spam:\n", spamFP)

Ham:
 Counter({'.': 18, ',': 15, 'the': 15, 'you': 10, 'to': 9, 'will': 6, 'a': 6, 'i': 5, 'of': 5, 'please': 4, 'is': 4, 'in': 4, 'and': 4, 'hi': 3, 'see': 3, 'soon': 3, 'dear': 3, 'my': 3, '?': 3, 'be': 3, 'with': 3, 'me': 3, 'have': 3, ':': 3, '2017': 3, 'from': 3, 'hans': 2, 'our': 2, 'family': 2, 'all': 2, 'best': 2, 'sue': 2, 'ata': 2, 'email': 2, 'related': 2, 'car': 2, 'insurance': 2, 'would': 2, 'discuss': 2, 'call': 2, 'if': 2, 'john': 2, 'just': 2, 'first': 2, 'present': 2, 'peter': 2, 'event': 2, 'studies': 2, 'artist': 2, 'wojtek': 2, 'sawa': 2, 'on': 2, 'p.m.': 2, 'his': 2, 'meet': 2, 'yes': 2, 'got': 2, 'week': 2, 'hope': 1, 'at': 1, 'party': 1, 'when': 1, 'arrive': 1, 'did': 1, 'receive': 1, 'last': 1, 'offer': 1, 'happy': 1, 'details': 1, 'give': 1, 'any': 1, 'questions': 1, 'smith': 1, 'super': 1, 'everyone': 1, 'this': 1, 'gentle': 1, 'reminder': 1, 'today': 1, "'s": 1, 'sls': 1, 'colloquium': 1, '2.30': 1, '4.00': 1, 'pm': 1, 'ballantine': 1, '103.': 1, 'rodica': 1,

In [12]:
from math import log

tokenlist = []
frqprofiles = []
for x in spam:
    frqprofiles.append( Counter(word_tokenize(x.lower())) )
    tokenlist.append( set(word_tokenize(x.lower())) )
for x in ham:
    frqprofiles.append( Counter(word_tokenize(x.lower())) )
    tokenlist.append( set(word_tokenize(x.lower())) )
#print(tokenlist)

for x in frqprofiles[0]:
    frq = frqprofiles[0][x]
    counter = 0
    for y in tokenlist:
        if x in y:
            counter += 1
    print(x, frq * log(len(tokenlist)/counter, 2))


our 6.339850002884625
medicine 6.339850002884625
cures 3.1699250014423126
baldness 3.1699250014423126
. 0.0
no 6.339850002884625
diagnostics 3.1699250014423126
needed 3.1699250014423126
we 4.754887502163469
guarantee 3.1699250014423126
fast 3.1699250014423126
viagra 6.339850002884625
delivery 3.1699250014423126
v1agra 3.1699250014423126
can 4.339850002884625
provide 3.1699250014423126
human 3.1699250014423126
growth 3.1699250014423126
hormone 3.1699250014423126
the 0.7251401587694168
cheapest 3.1699250014423126
life 3.1699250014423126
insurance 2.1699250014423126
with 3.5097750043269373
us 2.1699250014423126
you 0.33985000288462475
lose 3.1699250014423126
weight 6.339850002884625
this 3.1699250014423126
treatment 6.339850002884625
now 3.1699250014423126
and 4.6797000057692495
medical 3.1699250014423126
exams 3.1699250014423126
necessary 3.1699250014423126
online 2.1699250014423126
pharmacy 3.1699250014423126
is 2.1699250014423126
best 1.5849625007211563
cream 3.1699250014423126
removes

The probability that we pick randomly an e-mail that is spam or ham can be computed as the ratio of the counts divided by the number of e-mails:

In [13]:
total = len(spam) + len(ham)

spamP = len(spam) / total
hamP  = len(ham) / total

print("probability to pick spam:", spamP)
print("probability to pick  ham:", hamP)

probability to pick spam: 0.2222222222222222
probability to pick  ham: 0.7777777777777778


We will need the total token count to calculate the relative frequency of the tokens, that is to generate likelihood estimates. We could *brute force* add one to create space in the probability mass for unknown tokens.

In [14]:
totalSpam = sum(spamFP.values()) + 1
totalHam  = sum(hamFP.values()) + 1

print("total spam counts + 1:", totalSpam)
print("total  ham counts + 1:", totalHam)

total spam counts + 1: 147
total  ham counts + 1: 321


We can relativize the counts in the frequency profiles now:

In [15]:
hamFP  = Counter( dict([ (token, frequency/totalHam)  for token, frequency in hamFP.items() ]) )
spamFP = Counter( dict([ (token, frequency/totalSpam) for token, frequency in spamFP.items() ]) )

print(hamFP)
print("-" * 30)
print(spamFP)

Counter({'.': 0.056074766355140186, ',': 0.04672897196261682, 'the': 0.04672897196261682, 'you': 0.03115264797507788, 'to': 0.028037383177570093, 'will': 0.018691588785046728, 'a': 0.018691588785046728, 'i': 0.01557632398753894, 'of': 0.01557632398753894, 'please': 0.012461059190031152, 'is': 0.012461059190031152, 'in': 0.012461059190031152, 'and': 0.012461059190031152, 'hi': 0.009345794392523364, 'see': 0.009345794392523364, 'soon': 0.009345794392523364, 'dear': 0.009345794392523364, 'my': 0.009345794392523364, '?': 0.009345794392523364, 'be': 0.009345794392523364, 'with': 0.009345794392523364, 'me': 0.009345794392523364, 'have': 0.009345794392523364, ':': 0.009345794392523364, '2017': 0.009345794392523364, 'from': 0.009345794392523364, 'hans': 0.006230529595015576, 'our': 0.006230529595015576, 'family': 0.006230529595015576, 'all': 0.006230529595015576, 'best': 0.006230529595015576, 'sue': 0.006230529595015576, 'ata': 0.006230529595015576, 'email': 0.006230529595015576, 'related': 0.

We can now compute the default probability that we want to assign to unknown words as $1 / totalSpam$ or $1 / totalHam$ respectively. Whenever we encounter an unknown token that is not in our frequency profile, we will assign the default probability to it.

In [16]:
defaultSpam = 1 / totalSpam
defaultHam  = 1 / totalHam

print("default spam probability:", defaultSpam)
print("default  ham probability:", defaultHam)

default spam probability: 0.006802721088435374
default  ham probability: 0.003115264797507788


We can test an unknown document by calculating how likely it was generated by the hamFP-distribution or the spamFP-distribution. We have to tokenize the lower-cased unknown document and compute the product of the likelihood of every single token in the text. We should scale this likelihood with the likelihood of randomly picking a ham or a spam e-mail. Let us calculate the likelihood that the random email is spam:

In [21]:
unknownEmail = """Dear ,
we sell the cheapest and best Viagra on the planet. Our delivery is guaranteed confident and cheap.
"""
#unknownEmail = """Dear Hans,
#I have not seen you for so long. When will we go out for a coffee again.
#"""

tokens = word_tokenize(unknownEmail.lower())

result = 1.0
for token in tokens:
    result *= spamFP.get(token, defaultSpam)

print(result * spamP)

1.4169476187860514e-40


Since this number is very small, a better strategy might be to sum up the log-likelihoods:

In [22]:
from math import log

resultSpam = 0.0
for token in tokens:
    resultSpam += log(spamFP.get(token, defaultSpam), 2)
resultSpam += log(spamP)

print(resultSpam)

-131.70848976462216


In [23]:
resultHam = 0.0
for token in tokens:
    resultHam += log(hamFP.get(token, defaultHam), 2)
resultHam += log(hamP)

print(resultHam)

-144.4608493674179


The log-likelihood for spam is larger than for *ham*. Our simple classifier would have guessed that this e-mail is *spam*.

In [24]:
if max(resultHam, resultSpam) == resultHam:
    print("e-mail is ham")
else:
    print("e-mail is spam")

e-mail is spam


The are numerous ways to improve the algorithm and tutorial. Please [send me](http://cavar.me/damir/) your suggestions.

(C) 2017-2025 by [Damir Cavar](http://damir.cavar.me/) - [Creative Commons Attribution-ShareAlike 4.0 International License](https://creativecommons.org/licenses/by-sa/4.0/) ([CA BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/))